In [39]:
import re
import time
import numpy as np

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from profanityfilter import ProfanityFilter

import pandas as pd
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', 300)

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/becode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/becode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
raw = pd.read_csv("mails_raw.csv", index_col=0)
# Remove nan
raw = raw.drop(raw[raw["To"].isnull() & raw["X-To"].isnull()].index)

In [270]:
idx = raw[raw.Content.apply(lambda x: "confidential" in x)].From.index#value_counts().sum()

In [ ]:
for line in raw.loc[idx].Content.iloc[6].split("\n"):
    print(line)

In [3]:
df = raw[["Content", "Date"]]
df["Between"] = raw.apply(lambda x: [x["From"], x["To"]], axis=1)

/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
start = time.time()

tokenizer = RegexpTokenizer(r'\w+')

df["Token"] = df["Content"].apply(tokenizer.tokenize)
df["Token"] = df["Token"].apply(lambda x: [word.lower() for word in x])

print("Took", round(time.time()-start, 2), "seconds.")

/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Took 13.87 seconds.


/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
start = time.time()

stop_words = set(stopwords.words('english'))
df["Strip"] = df["Token"].apply(lambda x: [word for word in x if word not in stop_words])

print("Took", round(time.time()-start, 2), "seconds.")

Took 4.17 seconds.


/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
start = time.time()

df["Freq"] = df["Strip"].apply(lambda x: dict(FreqDist(x).most_common()))

print("Took", round(time.time()-start, 2), "seconds.")

Took 14.03 seconds.


/home/becode/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df.to_csv("mails_features.csv")

In [4]:
df = pd.read_csv("mails_features.csv", index_col=0)

In [497]:
raw

,Content,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName
0,"---------------------- Forwarded by John Arnold/HOU/ECT on 05/10/2001 02:21 \nPM ---------------------------\n\n\n""Schaefer, Matthew"" <MSchaefer@NYMEX.com> on 05/10/2001 10:52:30 AM\nTo: Brad Banky <bbanky@txues.com>, David Rosenberg \n<david.m.rosenberg@enron.com>, George Gero <anthony_gero@pru...",<25702046.1075849632104.JavaMail.evans@thyme>,2001-05-10 07:16:00-07:00,john.arnold@enron.com,mike.maggi@enron.com,Options Advisory Committee Meeting - May 31st,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,John Arnold,Mike Maggi,NaN,NaN,\John_Arnold_Oct2001\Notes Folders\Sent,ARNOLD-J,jarnold.nsf
1,David:\nDo you have a simulation set up that will allow me to simulate trades by \npushing a buy or sell button and will move the two way so i can try \nsimulating actual market cicumstances?\n\n\nFrom: David Forster/ENRON@enronXgate on 05/08/2001 09:55 AM\nTo: John Arnold/HOU/ECT@ECT\ncc: Savit...,<28095741.1075849632148.JavaMail.evans@thyme>,2001-05-13 07:52:00-07:00,john.arnold@enron.com,david.forster@enron.com,Re:,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,John Arnold,David Forster,NaN,NaN,\John_Arnold_Oct2001\Notes Folders\Sent,ARNOLD-J,jarnold.nsf
2,"Yolanda\n\nI will be attending the ski outing - I am not sure about my arrival and \ndeparture dates, but I did put tentative dates. Once I make my bookings I \nwill update you on my final dates. I might stay an extra day, and will pick \nthat up. Thanks\n\n\nHarry\n\n\n",<3676806.1075848347249.JavaMail.evans@thyme>,2000-12-15 03:44:00-08:00,harry.arora@enron.com,yolanda.roberts@enron.com,Re: Aspen,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Harry Arora,Yolanda Roberts,NaN,NaN,\Harpreet_Arora_Nov2001\Notes Folders\Sent,ARORA-H,harora.nsf
3,"Suresh\n\nFor the next two weeks, I'd like you to focus on the following.\n\n1. Development frame-work and getting Zach upto speed\n Specs for Syndication report & testing & production date\n Finalize questionnaire and production date\n Modifications to Invitee and User stats\n2. EnronOnline auc...",<22378087.1075848347271.JavaMail.evans@thyme>,2001-01-05 08:24:00-08:00,harry.arora@enron.com,suresh.raghavan@enron.com,Tasks,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Harry Arora,Suresh Raghavan,NaN,NaN,\Harpreet_Arora_Nov2001\Notes Folders\Sent,ARORA-H,harora.nsf
4,Dave\n\nFind enclosed the business plan and the valuation model for DealBench. Feel \nfree to call me with questions.\n\n\n\n\n\n\nHarry Arora,<30442669.1075848347294.JavaMail.evans@thyme>,2001-01-22 10:36:00-08:00,harry.arora@enron.com,dave.samuels@enron.com,Business Plan,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Harry Arora,Dave Samuels,NaN,NaN,\Harpreet_Arora_Nov2001\Notes Folders\Sent,ARORA-H,harora.nsf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125677,"\n\n -----Original Message-----\nFrom: \tWhiting, Greg \nSent:\tFriday, December 07, 2001 3:01 PM\nTo:\tZipper, Andy\nCc:\tHodges, Georgeanne\nSubject:\tCOF Curves for Andy Zipper.xls\n\nAndy,\n\nHere is the curves that were used to calc the 9/30 Cost of Funds for EWS.\n\n\t\t\t\tGreg x35399\n\n",<23879799.1075859061488.JavaMail.evans@thyme>,2001-12-10 06:36:33-08:00,andy.zipper@enron.com,david.oconnor@ubsw.com,FW: COF Curves for Andy Zipper.xls,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Zipper, Andy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>",'david.oconnor@ubsw.com',NaN,NaN,"\Andrew_Zipper_Jan2002\Zipper, Andy\Sent Items",Zipper-A,azipper (Non-Privileged).pst
125678,"\n\n--------- Inline attachment follows ---------\n\nFrom: </O=ENRON/OU=NA/CN=RECIPIENTS/CN=AZIPPER>\nTo: 'david.oconnor@ubsw.com'\nDate: Monday, December 10, 2001 2:36:33 GMT\nSubject: \n\n\n\n -----Original Message-----\nFrom: \tWhiting, Greg \nSent:\tFriday, December 07, 2001 3:01 PM\nT...",<3235534.1075859061510.JavaMail.evans@thyme>,2001-12-10 09:01:09-08:00,andy.zi

In [11]:
blb.TextBlob(df.loc[4, "Content"]).sentiment

Sentiment(polarity=0.4, subjectivity=0.8)

In [14]:
congrats_idx = raw[raw["Subject"] == "Congratulations"].reset_index()

In [28]:
idx = 0

In [32]:
print(blb.TextBlob(congrats_idx.loc[idx, "Content"],
             analyzer=NaiveBayesAnalyzer()).sentiment)
idx += 1

Sentiment(classification='pos', p_pos=0.9999768794496625, p_neg=2.312055034150871e-05)


In [127]:
fired = df[df["Content"].apply(lambda x: ("fired" in x) & ("fired up" not in x) & ("not fired" not in x))]
fired.drop(columns=[c for c in fired.columns if c != "Content"], inplace=True)
fired.reset_index(inplace=True)

In [149]:
raw[raw["Content"].apply(lambda x: "ljm" in x)]#.loc[85131,"Content"]

,Content,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName
11780,"Will be on vacation but wish MK the best of luck. Rick\n\n -----Original Message-----\nFrom: =09Dauterive, Inez =20\nSent:=09Monday, August 13, 2001 1:34 PM\nTo:=09Rios, Amy; Fastow, Andrew; Yaeger, Anne; Yaeger, Anne; Schnapper, Bar=\nry; Glisan, Ben; Thomason, Benjamin; Tauzier, Betty; Brown, ...",<29761275.1075863363125.JavaMail.evans@thyme>,2001-08-13 12:05:38-07:00,rick.buy@enron.com,inez.dauterive@enron.com,RE: Happy Hour,NaN,1.0,text/plain; charset=us-ascii,quoted-printable,NaN,"Buy, Rick </O=ENRON/OU=NA/CN=RECIPIENTS/CN=RBUY>","Dauterive, Inez </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Notesaddr/cn=637aa25f-549840a0-86256531-75eecc>",NaN,NaN,"\RBUY (Non-Privileged)\Buy, Rick\Sent Items",Buy-R,RBUY (Non-Privileged).pst
41684,I'm sure Enron's stockholders would want Enron to join a filing to allow\nEnron to participate in discussions between the PX and the Chairman of FERC\n(and to oppose any ex parte communications) about whether the PX holds\nEnron's money. I would be interested in going to the discussion too.\n\n...,<15019380.1075860475686.JavaMail.evans@thyme>,2001-02-04 09:22:00-08:00,mary.hain@enron.com,dwatkiss@bracepatt.com,Returned mail: Host unknown (Name server: bracepat.com: host not\n found),NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Mary Hain,dwatkiss@bracepatt.com,NaN,NaN,\Mary_Hain_Aug2000_Jul2001\Notes Folders\'sent mail,Hain-M,mary-hain.nsf
48655,"\n--------------------------vince kaminski\n\n\n-----Original Message-----\nFrom: Kaminski, Vince J <Vince.J.Kaminski@ENRON.com>\nTo: LJM Litigation <ljm.litigation@ENRON.com>\nCC: Cooley, Jan <Jan.Cooley@ENRON.com>; Kaminski, Vince J <Vince.J.Kaminski@ENRON.com>\nSent: Fri Dec 28 15:12:02 2001\...",<21995338.1075840793313.JavaMail.evans@thyme>,2001-12-28 14:07:51-08:00,j.kaminski@enron.com,vkaminski@aol.com,FW: Raptors,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENTS/CN=VKAMINS>",'vkaminski@aol.com',NaN,NaN,\vkamins\Sent Items,KAMINSKI-V,vincent kaminski 1-30-02.pst
66232,"did you see the article in the wsj today on that ljm fund associated with ene?\n\n -----Original Message-----\nFrom: \tHal Lenhart <Hal.Lenhart@apollogrp.edu>@ENRON \nSent:\tThursday, October 18, 2001 10:07 AM\nTo:\tLenhart, Matthew\nSubject:\tRE:\n\nEvery thing is good here. Tom and I played g...",<7586303.1075854976404.JavaMail.evans@thyme>,2001-10-18 10:31:10-07:00,matthew.lenhart@enron.com,hal.lenhart@apollogrp.edu,RE:,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS/CN=MLENHAR>",'Hal Lenhart <Hal.Lenhart@apollogrp.edu>@ENRON',NaN,NaN,\MLENHAR (Non-Privileged)\Sent Items,Lenhart-M,MLENHAR (Non-Privileged).pst
78095,"Dear Dr. Fidas,\n\nI spoke with you a few days ago concerning my interest in the Sercurity Policy Studies program. I am going to be visiting the GW campus the afternoon of Thursday Jan 24th and would like to know if you would be available for a few minutes. Also, you had mentioned you may be a...",<8612526.1075841213581.JavaMail.evans@thyme>,2002-01-16 12:05:51-08:00,larry.may@enron.com,gfidas@gwu.edu,Lawrence May,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"May, Larry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=LMAY2>",'gfidas@gwu.edu',NaN,NaN,"\ExMerge - May, Larry\Sent Items",MAY-L,larry may 6-26-02.PST
78096,"2:15 would be good. My flight gets into Reagan at 12:00: if it is delayed I will give you a call. Thank you,\n\nLarry May\n\n -----Original Message-----\nFrom: \tgfidas <gfidas@gwu.edu>@ENRON On Behalf Of gfidas <gfidas@gwu.edu>\nSent:\tTuesday, January 22, 2002 9:19 AM\nTo:\tMay, Larry\nSu...",<1645591.1075841213557.JavaMail.evans@thyme>,2002-01-22 08:34:24-08:00,larry.may@enron.com,gfidas@gwu.edu,RE: Lawrence May,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"May, Larry </O=ENRON/OU=NA/CN=RECIPIENTS/CN=LMAY2>",'gfidas <gfidas@gwu.edu>

In [ ]:
"Bethany McLean" "Andy Fastow" "LJM"

In [ ]:
df["Sentiment"] = df["Content"].apply(lambda x: TextBlob(x).sentiment)
df["Profanity"] = df["Content"].apply(lambda x: pf.is_profane(x))

In [53]:
df.to_csv("mails_features.csv")

In [59]:
profanity = df[df["Profanity"] & df["Content"].apply(lambda x: "Gay" not in x)]
profanity.reset_index(inplace=True)
idx = 0

In [491]:
import spacy
import en_core_web_lg

eng = en_core_web_lg.load()
words = list(eng.vocab.strings)

In [189]:
import math
from numpy.linalg import norm

3.7416573867739413

In [191]:
def distance_vector(a, b):
    return math.sqrt(sum(
        [math.pow(b[idx] -a[idx], 2) for idx in range(len(a))]
))

def cos_sim(a, b):
    if norm(a) > 0 and norm(b) > 0:
        return np.dot(a, b) / norm(a) *norm(b)
    else: return 0.0

In [192]:
a = eng.vocab["cheddar"].vector
b = eng.vocab["cheese"].vector
distance_vector(a, b)

4.701113658674066

In [194]:
cos_sim(eng.vocab["puppy"].vector, eng.vocab["kitten"].vector)

34.815727

In [195]:
cos_sim(eng.vocab["trousers"].vector, eng.vocab["octopus"].vector)

6.836217

In [196]:
cos_sim(eng.vocab["milk"].vector, eng.vocab["cheese"].vector)

40.444374

In [219]:
from ast import literal_eval
import swifter
df["Token"] = df["Token"].swifter.apply(literal_eval)

In [496]:
start = time.time()
df.loc[:100, "Content"].apply(lambda x: TextBlob(x, analyzer=NaiveBayesAnalyzer()).sentiment)
print(time.time()-start)

544.6467547416687


In [404]:
def closest_words(tokens, vector, count=10, exclude=[]):
    result = sorted(
        tokens, reverse = True,
        key = lambda x: cos_sim(vector, eng.vocab[x].vector)
    )
    return [word for word in result if word not in exclude][:count]

closest_words(df.loc[17, "Token"], eng.vocab["money"].vector, count=3)

['job', 'good', 'it']

In [448]:
with open("chapter1.txt", "r") as file:
    text = file.read()

In [449]:
for mistake in list(filter(lambda x: x[-1].isupper(), re.findall(r".{1}\..{1}", text))):
    re.sub(mistake, mistake[:2] + " " +mistake[-1], text)

In [456]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
game = tokenizer.tokenize(text)

#game = list(map(lambda x: x.lower(), game))
tokens = []
for word in game:
    if word not in tokens:
        tokens.append(word)
tokens.append("queen")

In [459]:
a = "king"
b = "man"
c = "woman"
vector = eng.vocab[a].vector - eng.vocab[b].vector + eng.vocab[c].vector
#vector = eng.vocab["king"].vector
closest_words(tokens, vector, count=5, exclude=[a, b, c])

['King', 'queen', 'kings', 'ruler', 'Kingdoms']

In [431]:
"mouth" in tokens

True

In [429]:
text

'The morning had dawned clear and cold, with a crispness that hinted at the end of summer. They set forth at daybreak to see a man beheaded, twenty in all, and Bran rode among them, nervous with excitement. This was the first time he had been deemed old enough to go with his lord father and his brothers to see the king’s justice done. It was the ninth year of summer, and the seventh of Bran’s life.The man had been taken outside a small holdfast in the hills. Robb thought he was a wildling, his sword sworn to Mance Rayder, the King-beyond-the-Wall. It made Bran’s skin prickle to think of it. He remembered the hearth tales Old Nan told them. The wildlings were cruel men, she said, slavers and slayers and thieves. They consorted with giants and ghouls, stole girl children in the dead of night, and drank blood from polished horns. And their women lay with the Others in the Long Night to sire terrible half-human children.But the man they found bound hand and foot to the holdfast wall awaiti